<a href="https://colab.research.google.com/github/GalinaZh/Appl_alg2021/blob/main/Applied_Alg_sem_10_rank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Занятие 10 
# Прикладная алгебра и численные методы
## Матричные нормы. Приближение матриц меньшего ранга

In [5]:
import numpy as np
import sympy
import scipy.linalg

In [9]:
!python -m pip install scipy --upgrade

     |████████████████████████████████| 27.4MB 147kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
sympy.__version__, scipy.__version__, np.__version__

('1.7.1', '1.6.1', '1.19.5')

## Подчиненные (операторные) матричные нормы.

https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html

numpy.linalg.norm(x, ord=None, axis=None, keepdims=False)

https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.norm.html

scipy.linalg.norm(a, ord=None, axis=None, keepdims=False, check_finite=True)

если check_finite=True, проверяется, все ли элементы матрицы конечны  


#### Возможные значения параметра ord 
одинаково в numpy и scipy
$$
\begin{matrix}
ord &	norm\ for\ matrices\\
None &		Frobenius\ norm\\
‘fro’	 &	Frobenius\ norm\\
‘nuc’ &		nuclear\ norm\\
inf &		max(sum(abs(x),\ axis=1))\\
-inf &		min(sum(abs(x),\ axis=1))\\
0	 &	–\\
1	 &	max(sum(abs(x),\ axis=0))\\
-1 &		min(sum(abs(x),\ axis=0))\\
2	 &	2-norm\ (largest\ singular\ value)\\
-2 &		smallest\ singular\ value\\
other &		–
\end{matrix}
$$


Матричная норма 
$ \|A\|_{1}=\max \limits _{1\leq j\leq n}\sum _{i=1}^{m}|a_{ij}|$, подчинённая векторной норме$ \|x\|_{1}=\sum _{i=1}^{n}|x_{i}|$.

Матричная норма $\|A\|_{\infty }=\max \limits _{1\leq i\leq m}\sum _{j=1}^{n}|a_{ij}|$, подчинённая векторной норме $\|x\|_{\infty }=\max \limits _{1\leq i\leq n}|x_{i}|$.

Спектральная норма $\|A\|_{2}=\sup \limits _{\|x\|_{2}=1}\|Ax\|_{2}=\sup \limits _{(x,x)=1}{\sqrt {(Ax,Ax)}}={\sqrt {\lambda _{max}(A^{*}A)}}$, подчиненная векторной норме $\|x\|_{2}={\sqrt {\sum _{i=1}^{n}|x_{i}|^{2}}}$.

## Пример 1.
Вычислим матричные нормы для матрицы 
$$
\left(
\begin{matrix}
1 & 3 & 5\\
2 & 5 & 8\\
3 & 7 & 11
\end{matrix}
  \right)
$$


In [34]:
M = [[1, 3, 5], [2, 5, 8], [3, 7, 11]]
for package in (np, scipy):
  display(package.__name__, *[('ord', item, '||M||', package.linalg.norm(M, ord=item)) for item in (1, 2, np.inf)])

'numpy'

('ord', 1, '||M||', 24.0)

('ord', 2, '||M||', 17.518067568404707)

('ord', inf, '||M||', 21.0)

'scipy'

('ord', 1, '||M||', 24.0)

('ord', 2, '||M||', 17.518067568404707)

('ord', inf, '||M||', 21.0)

https://docs.sympy.org/latest/modules/matrices/matrices.html#sympy.matrices.matrices.MatrixBase.norm

В sympy реализованы нормы с такими же значениями ord, как в scipy и numpy:

norm(ord=None)

это метод в классе матриц.

## Пример 2
Вычислим нормы $||M||_1$, $||M||_2$ и $||M||_\infty$  матрицы Примера 1 с помощью sympy:


In [42]:
Msym = sympy.Matrix(M)
display([('ord', item, '||M||', [(Mnorm, round(Mnorm, 5)) for Mnorm in [Msym.norm(ord=item)]]) for  item in (1, 2, sympy.oo)])

[('ord', 1, '||M||', [(24, 24)]),
 ('ord', 2, '||M||', [(sqrt(sqrt(94105)/2 + 307/2), 17.51807)]),
 ('ord', oo, '||M||', [(21, 21)])]

## Пример 3


## Пример 4


(0.009465126280781055, None)
(0.10277248496370375, None)
(0.10357010563457898, None)





## Пример 5



(0.10277248496370375, 0.012283176171385676)
(0.1036058697929914, 0.000833384829287645)
(0.10357010563457898, 3.576415841241576e-05)
